In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC, LinearSVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, log_loss
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.calibration import CalibratedClassifierCV
from sklearn.externals import joblib
import pickle
import jieba

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def save_model(Vectorizer, Vectorizer_filename, Model, Model_filename):
    print('Save model...')
    print(joblib.dump(Vectorizer, 'model/' + Vectorizer_filename + '.model'))
    print(joblib.dump(Model, 'model/'+ Model_filename +'.model'))

In [3]:
df_train = pd.read_csv('data/train_after_preprocess.csv')
df_train.head()

,ID,review,label
0,1,jo bhi ap se tou behtar hoon,Negative
1,2,ya allah meri sister affia ki madad farma,Positive
2,3,yeh khud chahta umar main shadi krna ogi,Negative
3,4,tc apky mun xe exe alfax achy nae lgty,Negative
4,5,good,Positive


In [4]:
print(len(df_train['review']))
print(type(df_train['review']))

6328
<class 'pandas.core.series.Series'>


In [5]:
df_test = pd.read_csv("data/test_after_preprocess.csv")
df_test.head()

,ID,review
0,1,phr tissuw se saaf
1,2,jail road per firing se shakhs janbahaq
2,3,mehfil loot li aunty ne
3,4,rehnay butt sahb nay galiya boht deni hain
4,5,zabardast


In [6]:
print(len(df_test['review']))
type(df_test['review'])

2712


pandas.core.series.Series

In [7]:
X = np.array([review for review in df_train['review']])
Y = np.array([1 if label == 'Positive' else 0 for label in df_train['label']])
X_test = np.array([review for review in df_test['review']])

In [8]:
print(X.shape)
print(X_test.shape)
X_all = np.concatenate((X, X_test), axis = 0)
print(X_all.shape)

(6328,)
(2712,)
(9040,)


In [9]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, shuffle=True)
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)

(4429,)
(4429,)
(1899,)
(1899,)


In [10]:
Counter = CountVectorizer(min_df=3, max_df=0.5, ngram_range=(1,2))
Counter.fit(X_all)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=3,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
X_train_vec = Counter.transform(X_train)
X_valid_vec = Counter.transform(X_valid)
X_train_all = Counter.transform(X)
print(X_train_vec.shape)
print(Y_train.shape)
print(X_valid_vec.shape)
print(X_valid.shape)
print(X_train_all.shape)
print(Y.shape)

(4429, 10693)
(4429,)
(1899, 10693)
(1899,)
(6328, 10693)
(6328,)


In [12]:
clf =  CalibratedClassifierCV(LinearSVC(), cv = 5) 
clf.fit(X_train_vec, Y_train)

CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=5, method='sigmoid')

In [13]:
print('The ACC on train set is ' + str(clf.score(X_train_vec, Y_train)))
Y_train_predict = clf.predict_proba(X_train_vec)
Y_train_predict_postive = np.array([item[1] for item in Y_train_predict])
print('AUC = ' + str(roc_auc_score(Y_train, Y_train_predict_postive)))

print('The ACC on test set is ' + str(clf.score(X_valid_vec, Y_valid)))
Y_valid_predict = clf.predict_proba(X_valid_vec)
Y_valid_predict_postive = np.array([item[1] for item in Y_valid_predict])
print('AUC = ' + str(roc_auc_score(Y_valid, Y_valid_predict_postive)))

The ACC on train set is 0.9932264619552946
AUC = 0.9998497067133865
The ACC on test set is 0.7309110057925223
AUC = 0.8079669746500274


In [14]:
clf = MultinomialNB()
clf.fit(X_train_vec, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
print('The ACC on train set is ' + str(clf.score(X_train_vec, Y_train)))
Y_train_predict = clf.predict_proba(X_train_vec)
Y_train_predict_postive = np.array([item[1] for item in Y_train_predict])
print('AUC = ' + str(roc_auc_score(Y_train, Y_train_predict_postive)))

print('The ACC on test set is ' + str(clf.score(X_valid_vec, Y_valid)))
Y_valid_predict = clf.predict_proba(X_valid_vec)
Y_valid_predict_postive = np.array([item[1] for item in Y_valid_predict])
print('AUC = ' + str(roc_auc_score(Y_valid, Y_valid_predict_postive)))

The ACC on train set is 0.8832693610295778
AUC = 0.956865315540244
The ACC on test set is 0.760926803580832
AUC = 0.8238026076172813


In [16]:
X_test_vec = Counter.transform(X_test)
Y_predict = clf.predict_proba(X_test_vec)
Y_predict_positive = [item[1] for item in Y_predict]
test_ids = df_test['ID']
Data = {'ID':test_ids, 'Pred':Y_predict_positive}
result = pd.DataFrame(Data, columns=['ID', 'Pred'])
result.to_csv('data/test_pred.csv', header = True)
result.head()

,ID,Pred
0,1,4.930876e-01
1,2,1.705631e-07
2,3,2.013359e-01
3,4,3.388344e-02
4,5,7.816524e-01
